# Prepare Env

## Pre Requisites

- 3 Linux machines with PostgreSQL installed that are reachable for each other network wise
- Hosts named resolved with db-1, db-2, db-3

# Demo

## Adjust Configurations

In [ ]:
#### On Every Machine ####
su postgres
cd $PGDATA
## WAL Level ##
sed -i "s/#*wal_level.*/wal_level = logical/" postgresql.conf

## Logs ##
sed -i "s/#*logging_collector.*/logging_collector = on/" postgresql.conf
sed -i "s/#*log_replication_commands.*/log_replication_commands = on/" postgresql.conf
sed -i "s/#*log_min_messages.*/log_min_messages = INFO/" postgresql.conf
alias pg_logs='tail -n 20 log/`ls -t log | head -n 1`'


## Restart server to apply configurations
pg_ctl restart
psql -c "SHOW wal_level;" # Make sure it's logical

## Single Source

### Setup Replication

#### Create Tables

In [ ]:
#### On Both db-1, db-2 ####
psql -c "CREATE TABLE replicated_a(id INT PRIMARY KEY, data VARCHAR(40));"
psql -c "CREATE TABLE replicated_b(id INT PRIMARY KEY, data VARCHAR(40));"

#### Create Publication

In [ ]:
#### On db-1 ####
psql -c "CREATE PUBLICATION pub_all_tbl FOR ALL TABLES;"
psql -c "SELECT * FROM pg_publication;"
psql -c "SELECT * FROM pg_publication_tables;"

#### Create Subscription

In [ ]:
#### On db-2 ####
psql -c "CREATE SUBSCRIPTION sub_db1_all_tbl CONNECTION 'host=db-1 password=cool-postgres-course' PUBLICATION pub_all_tbl;"
psql -c "SELECT * FROM pg_subscription;"
psql -c "SELECT * FROM pg_subscription_rel;"
pg_logs

#### Check out what happend on db-1 as well ##
pg_logs -n 60

### Change Rows

#### Set select alias

In [ ]:
#### db-2 ####
alias pg_select_rep_a='psql -c "SELECT * FROM replicated_a;"'
alias pg_select_rep_b='psql -c "SELECT * FROM replicated_b;"'

#### Insert

In [ ]:
#### db-1 ####
psql -c "INSERT INTO replicated_a VALUES (1, 'a'), (2, 'b');"

#### db-2 ####
pg_select_rep_a

#### Update

In [ ]:
#### db-1 ####
psql -c "UPDATE replicated_a SET data = 'c' WHERE id = 1;"

#### db-2 ####
pg_select_rep_a

#### Delete

In [ ]:
#### db-1 ####
psql -c "DELETE FROM replicated_a WHERE id = 2;"

#### db-2 ####
pg_select_rep_a

### Handle Conflicts

#### Set monitoring alias

In [ ]:
#### db-2 ####
alias pg_sub_stats='echo "SELECT * FROM pg_stat_subscription_stats; \watch i=5 c=4" | psql' # Cumulative
alias pg_sub='psql -c "SELECT * FROM pg_stat_subscription;"' # Dynamic

#### Create Conflict

In [ ]:
#### db-2 ####
psql -c "ALTER TABLE replicated_a ADD CONSTRAINT a_unique_data UNIQUE(data);"

#### db-1 ####
psql -c "INSERT INTO replicated_a VALUES (2, 'z'), (3, 'z');"

#### db-2 ####
## Check out the subscription
pg_select_rep_a
pg_logs -n 100
pg_sub
pg_sub_stats


#### db-1 ####
## Look on the delta
psql << EOM
SELECT 
    pg_size_pretty(pg_current_wal_lsn() - confirmed_flush_lsn) fall_behaind_size
FROM pg_replication_slots
WHERE slot_name = 'sub_db1_all_tbl';
EOM

#### Disable Subscription - Quit Loop

In [ ]:
#### db-2 ####
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl SET (DISABLE_ON_ERROR='on');"

## Check out the subscription
pg_sub
pg_sub_stats

#### Handle By Changing Constraint

In [ ]:
#### db-2 ####
## Remove constraint to handle error
psql -c "ALTER TABLE replicated_a DROP CONSTRAINT a_unique_data;"

## Enable the subscription
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl ENABLE;"
pg_select_rep_a

#### Handle By Skipping LSN

In [ ]:
###### Create conflict on replicated_b ######
#### db-2 ####
psql -c "ALTER TABLE replicated_b ADD CONSTRAINT b_unique_data UNIQUE(data);"

#### db-1 ####
psql -c "INSERT INTO replicated_b VALUES (2, 'z'), (3, 'z');"
psql -c "INSERT INTO replicated_b VALUES (4, 'z'), (5, 'x');"


###### Handle by skipping LSN ######
## Find problematic LSN
pg_logs

## Skip LSN
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl SKIP (LSN='<lsn>');"

## Enable the subscription
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl ENABLE;"
pg_select_rep_b
pg_sub
pg_sub_stats

## Multiple Sources

### Setup Replication

In [ ]:
#### db-3 ####
psql -c "CREATE TABLE replicated_a(id INT PRIMARY KEY, data VARCHAR(40));"
psql -c "CREATE PUBLICATION pub_rep_a FOR TABLE replicated_a;"

#### db-2 ####
psql -c "CREATE SUBSCRIPTION sub_db3_rep_a CONNECTION 'host=db-3 password=cool-postgres-course' PUBLICATION pub_rep_a;"

### Change Rows

In [ ]:
#### db-3 ###
psql -c "INSERT INTO replicated_a VALUES (6, 'f'), (7, 'g');"

#### db-2 ###
pg_select_rep_a